In [1]:
# 필요한 라이브러리 다시 임포트
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 데이터 파일 다시 읽기
competition_info = pd.read_csv('C:/Users/82106/Desktop/데이터분석 프로젝트/데이콘 경진대회/data/open/competition_info.csv')
participate_log = pd.read_csv('C:/Users/82106/Desktop/데이터분석 프로젝트/데이콘 경진대회/data/open/participate_log.csv')
submission_log = pd.read_csv('C:/Users/82106/Desktop/데이터분석 프로젝트/데이콘 경진대회/data/open/submission_log.csv')
talk_log = pd.read_csv('C:/Users/82106/Desktop/데이터분석 프로젝트/데이콘 경진대회/data/open/talk_log.csv')
codeshare_log = pd.read_csv('C:/Users/82106/Desktop/데이터분석 프로젝트/데이콘 경진대회/data/open/codeshare_log.csv')

# 대회별 참가자 수, 제출 수, 토크 수, 코드 공유 수 집계
participate_counts = participate_log.groupby('ID')['참가자 수'].sum()
submission_counts = submission_log.groupby('ID')['제출 수'].sum()
talk_counts = talk_log.groupby('ID').agg({'토크 수':'sum', '댓글 수':'sum'})
codeshare_counts = codeshare_log.groupby('ID')['코드 공유 수'].sum()

# 모든 데이터를 하나의 데이터프레임으로 합치기
combined_df = pd.DataFrame({
    '참가자 수': participate_counts,
    '제출 수': submission_counts,
    '토크 수': talk_counts['토크 수'],
    '댓글 수': talk_counts['댓글 수'],
    '코드 공유 수': codeshare_counts
}).reset_index()

# competition_info와 결합하여 대회명과 다른 정보 포함
trend_analysis_df = pd.merge(combined_df, competition_info[['ID', '대회명', '시작시간', '종료시간']], on='ID', how='left')

# 연도별 평균 참가자 수, 제출 수, 토크 및 코드 공유 수 계산
yearly_averages = trend_analysis_df.groupby(pd.to_datetime(trend_analysis_df['시작시간']).dt.year).mean()

# 특성과 타겟 데이터 준비 (연도별 통계만 사용)
features = yearly_averages[['참가자 수', '제출 수', '토크 수', '코드 공유 수']].values
target = yearly_averages.index.values  # 연도를 타겟으로 사용

### 대회 키워드 데이터를 사용하여 주제 모델링을 수행

##### LDA(Latent Dirichlet Allocation) 모델을 사용하여 대회 키워드 데이터에서 추출한 상위 5개의 주제와 각 주제에 해당하는 상위 10개 키워드

In [2]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# 주제 모델링을 위한 데이터 준비
# 대회 키워드 데이터
keywords_data = competition_info['키워드'].dropna()

# CountVectorizer를 사용하여 텍스트 데이터를 수치화
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(keywords_data)

# LDA 모델 생성 및 학습
# 여기서는 예시로 주제의 수를 5로 설정
lda = LatentDirichletAllocation(n_components=5, random_state=0)
lda.fit(X)

# 주제 별 상위 단어 추출
def get_top_words(model, feature_names, n_top_words):
    top_words = {}
    for topic_idx, topic in enumerate(model.components_):
        top_words[topic_idx] = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    return top_words

top_words = get_top_words(lda, vectorizer.get_feature_names_out(), 10)
top_words


{0: ['시계열', 'weighted', '비전', '생육', 'rmse', 'mae', '분석', 'f1', '산업', '아이디어'],
 1: ['분류',
  '알고리즘',
  'accuracy',
  'logloss',
  'nlp',
  '자연어',
  'audio',
  '과학',
  '음향',
  '유사성'],
 2: ['정형', '알고리즘', '회귀', '정성평가', '금융', '분석시각화', '분석', 'rmse', 'mae', 'nmae'],
 3: ['알고리즘', '분류', 'f1', 'score', 'macro', '언어', '비전', '정형', '탐지', '생성요약'],
 4: ['cv', '알고리즘', '이미지', '제어', 'map', '데이크루', '서포터즈', '객체', '시계열', '검출']}

주제 0: 

'시계열', 'weighted', '비전', '생육', 'rmse', 'mae', '분석', 'f1', '산업', '아이디어'

이 주제는 시계열 데이터, 비전 분야, 정량적 평가 방법(RMSE, MAE), 아이디어와 관련된 산업 분야를 나타낼 수 있다.

주제 1: 

'분류', '알고리즘', 'accuracy', 'logloss', 'nlp', '자연어', 'audio', '과학', '음향', '유사성'

이 주제는 분류 알고리즘, 정확도 평가, 자연어 처리(NLP), 오디오 및 음향 데이터 처리에 초점을 맞춘 것으로 보인다.

주제 2: 

'정형', '알고리즘', '회귀', '정성평가', '금융', '분석시각화', '분석', 'rmse', 'mae', 'nmae'

이 주제는 정형 데이터, 회귀 분석, 금융 분야 및 데이터 분석과 시각화에 관련된 내용을 포함한다.

주제 3: 

'알고리즘', '분류', 'f1', 'score', 'macro', '언어', '비전', '정형', '탐지', '생성요약'

이 주제는 다양한 알고리즘, 특히 분류 및 언어, 비전 처리와 관련된 주제를 나타낸다.

주제 4:

'cv', '알고리즘', '이미지', '제어', 'map', '데이크루', '서포터즈', '객체', '시계열', '검출'



이 주제는 컴퓨터 비전(CV), 이미지 처리, 객체 탐지와 관련된 기술을 중점적으로 다루는 것으로 보인다.

이러한 분석을 통해, 데이콘 대회가 앞으로 컴퓨터 비전, 자연어 처리, 금융 분석, 시계열 데이터 처리 등 다양한 분야를 포괄하는 대회를 개최할 

가능성이 높다는 것을 예측할 수 있다. 